In [ ]:
# Libraries

from scrapping import get_data
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [ ]:
# Functions

def count_skips(df):
    counts = {key: 0 for key in numbers}
    for columns in df:
        contador = 0
        for i in reversed(df[columns]):
            if i is not True:
                contador += 1
            else:
                counts[columns] = contador
                break
    return counts

In [ ]:
# Load the data base and obtain the first DataFrame
db = pd.read_parquet('db.parquet')
winners = db.iloc[:, 2:7]

# Search in every row and convert it to another DataFrame with boolean results, where True means that the number won, and False means that it loose
for e in range(1, 6):
    for i in range(1, 51):
        winners.loc[winners[f"Nro{e}"] == i, f"{i}"] = True

winners.drop(columns = ["Nro1","Nro2","Nro3","Nro4","Nro5"], inplace = True)

    # Add an extra row, in order to compare the number that did not appear for the first time in the game history
    
d_0 = pd.DataFrame(columns = [str(i) for i in range (1, 51)], index = [0]).fillna(True)

# Create the final DataFrame
winners = pd.concat([d_0, winners]).reset_index(drop = True)

In [ ]:
draws = list(np.arange(1, len(winners)))
numbers = [str(i) for i in range(1, 51)]
dicts = {draw: {key:[] for key in numbers} for draw in draws}

for e in draws:
    df = winners.loc[:e]
    counts = count_skips(df)
    dicts[e].update(counts)

skip_numbers = pd.DataFrame(dicts).T

In [ ]:
# Order the last draw for skips:
last_draw = sorted(dicts[len(skip_numbers)].items(), key = lambda x: x[1])
last_draw = pd.DataFrame({'Numero': [x[0] for x in last_draw], 'Skips': [int(x[1]) for x in last_draw]})

# Select  the last 12 draws
last_12_draws = np.arange(len(skip_numbers) - 12, len(skip_numbers) + 1)
sk_12 = skip_numbers.loc[last_12_draws]

# Count the numbers that hasn´t appeared in the lasts draws
skips = np.arange(0, 19)
aus = []
last_7 = []
last_12 = []

for i in last_12_draws[1:13]:
    for column in sk_12:
        if sk_12.loc[i, str(column)] == 0:
            aus.append(sk_12.loc[i - 1, str(column)])

for number in skips:
    last_12.append(aus.count(number))
    last_7.append(aus[25:60].count(number))

# Create a DataFrame of the counts for skips in each of the last 7 and 12 draws
skips_7_12 = pd.DataFrame(np.column_stack([last_7, last_12]), columns = [7, 12])

In [ ]:
groups = [list(range(i, i + 10)) for i in range(1, 51, 10)]
group_names = [tuple(range(i, i + 10)) for i in range(1, 51, 10)]
groups_of_winners = db.iloc[:, 2:7]
results = {i: {group_name: sum([1 for num in row if num in group]) for group_name, group in zip(group_names, groups)} for i, row in groups_of_winners.iterrows()}
results_df = pd.DataFrame.from_dict(results, orient = 'index')
results_df.columns = ['{}_to_{}'.format(i, i + 9) for i in range(1, 51, 10)]
sg_10 = results_df.iloc[-10:]
sg_5 = results_df.iloc[-5:]
groups_df = pd.DataFrame({'10_games': (sg_10 > 0).sum(), '5_games': (sg_5 > 0).sum()}).T

In [ ]:
db

In [ ]:
skip_numbers

In [ ]:
last_draw

In [ ]:
skips_7_12

In [ ]:
results_df

In [ ]:
groups_df

In [ ]:
# values = list(set(val for dic in [ultimo_sorteo] for val in dic.values()))